In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler
from IPython.display import display

In [2]:
crawler = NewsCrawler()

# 1. URL을 모은다

In [3]:
# foxnews, foxbusiness를 합치자. 다른 소스도 집어넣자...

In [4]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (51.91 seconds): URLs collecting... aljazeera           

n_total     11209
n_unique    11175
dtype: int64

huffpost               66
cnn                   819
investingcom          988
politico              219
time                   19
cnbc                  163
foxnews               200
foxbusiness           132
bbc                   291
businessinsider       612
morningstar            86
wsj                   106
nyt                    97
guardian              155
reuters              1153
washingtontimes       449
washingtonpost        134
cbs                   401
marketwatch           140
atlantic               96
vice                   17
npr                   851
newrepublic            20
yahoo                 232
independent           716
heritage              282
zdnet                 380
townhall              609
abcnews               100
hotair                 86
cbc                   299
nymag                 233
thestreet             142
thinkprogress          82
dailybeast             51
realclearpolitics     299
aljazeera             484
dtype: int64

,pubs
https://cbsnews.com/live-news/trump-impeachment-inquiry-latest-updates-today-2019-10-02,"cbs, dailybeast"
https://foxbusiness.com/business-leaders/nike-ceo-mark-parker-doping-scandal,"foxnews, foxbusiness"
https://foxbusiness.com/economy/mortgage-applications-jump-as-rates-fall,"foxnews, foxbusiness"
https://foxbusiness.com/lifestyle/pampered-chef-offers-new-healthy-meal-enhancements,"foxnews, foxbusiness"
https://foxbusiness.com/markets/chick-fil-a-vp-dee-ann-turner-remarkable-corporate-culture,"foxnews, foxbusiness"
https://foxbusiness.com/markets/hotel-resort-worker-ubs-banker-cocaine,"foxnews, foxbusiness"
https://foxbusiness.com/markets/is-there-actually-a-work-life-balance,"foxnews, foxbusiness"
https://foxbusiness.com/money/dave-ramsey-seven-steps-to-living-the-life-of-your-dreams,"foxnews, foxbusiness"
https://foxbusiness.com/money/holiday-sales-expected-to-hit-record-136-billion-this-year-despite-shorter-season,"foxnews, foxbusiness"
https://foxbusiness.com/money/hottest-us-cities-real-estate-money-and-why,"foxnews, foxbusiness"


# 2. 모은 URL중 이미 받은 것은 버린다

In [5]:
select_results = crawler.select(); display(*select_results)

100.00% (1.10 seconds): URLs selecting... aljazeera           

n_total     2137
n_unique    2122
dtype: int64

huffpost              13
cnn                  102
investingcom         267
politico              23
time                   6
cnbc                  26
foxnews               70
foxbusiness           26
bbc                   41
businessinsider      172
morningstar           17
wsj                   32
nyt                   32
guardian              69
reuters              454
washingtontimes       48
washingtonpost        53
cbs                   24
marketwatch           43
atlantic               5
vice                   0
npr                   13
newrepublic            3
yahoo                125
independent           99
heritage               1
zdnet                 49
townhall              22
abcnews               42
hotair                 7
cbc                   27
nymag                  7
thestreet             39
thinkprogress          0
dailybeast            30
realclearpolitics     31
aljazeera            119
dtype: int64

,pubs
https://cbsnews.com/live-news/trump-impeachment-inquiry-latest-updates-today-2019-10-02,"cbs, dailybeast"
https://foxbusiness.com/economy/mortgage-applications-jump-as-rates-fall,"foxnews, foxbusiness"
https://foxbusiness.com/markets/chick-fil-a-vp-dee-ann-turner-remarkable-corporate-culture,"foxnews, foxbusiness"
https://foxbusiness.com/markets/hotel-resort-worker-ubs-banker-cocaine,"foxnews, foxbusiness"
https://foxbusiness.com/markets/is-there-actually-a-work-life-balance,"foxnews, foxbusiness"
https://foxbusiness.com/money/dave-ramsey-seven-steps-to-living-the-life-of-your-dreams,"foxnews, foxbusiness"
https://foxbusiness.com/money/holiday-sales-expected-to-hit-record-136-billion-this-year-despite-shorter-season,"foxnews, foxbusiness"
https://foxbusiness.com/money/hottest-us-cities-real-estate-money-and-why,"foxnews, foxbusiness"
https://foxbusiness.com/money/how-a-michigan-couple-paid-off-120000-in-student-loans-in-less-than-3-years,"foxnews, foxbusiness"
https://foxbusiness.com/money/taxpayer-monthly-bill-shuttered-california-jail,"foxnews, foxbusiness"


# 3. 해당 URL의 뉴스를 다운로드한다

In [ ]:
download_results = crawler.download(); download_results # 뒤에 pub print는 빼도 되겠다...

45.95% (225.54 seconds): downloading...

In [8]:
crawler.urls_final['aljazeera']

{'downloaded': {'https://aljazeera.com/ajimpact/economic-crisis-deepens-zimbabwe-president-pleads-patience-191001133947141.html',
  'https://aljazeera.com/ajimpact/global-manufacturing-slowing-activity-decade-191001155910492.html',
  'https://aljazeera.com/ajimpact/titanic-shipyard-stay-afloat-191001130205332.html',
  'https://aljazeera.com/ajimpact/uaw-rejects-gm-offer-strike-spurs-6000-mexico-furloughs-191001210159056.html',
  'https://aljazeera.com/ajimpact/ups-drone-airline-government-launch-191001192509239.html',
  'https://aljazeera.com/ajimpact/warren-ocasio-cortez-probe-private-equity-firms-prison-ties-191001173725979.html',
  'https://aljazeera.com/indepth/inpictures/pictures-70-years-communist-china-191001063831523.html',
  'https://aljazeera.com/news/2019/10/1000-people-dead-mediterranean-year-191001140843136.html',
  'https://aljazeera.com/news/2019/10/baghdad-police-open-fire-anti-government-protesters-191001143248615.html',
  'https://aljazeera.com/news/2019/10/bayern-mun

In [10]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['aljazeera']['trashed']]

['634290012759c6575f9e6a0b3b3392615f6b4981',
 '86a337159e82bb76454c01fa0527bdcc13477579',
 'a8120f25106320e8e174168338a266987ee89014',
 'b4874a4e6bdb4c1ceb94b098271d33c14776c789',
 'ad9a88f10c1b719dc3858f8e056a4f2e2c2fbf99',
 'e01de0094525db57dca98ce3a277eef6c7b84b48',
 'a806f7260325f95500523cd3e11bd79b2fa5951a',
 'd8dddf47c77fb9e177e26c035af78f28038b2304',
 '4879782808dbb7768e13d96ab151fb20f6f5a03b',
 'b69b4bb3b881ba7a457d77070e907126b7aa72e1',
 'f07fe77d2f34b9c59824485370228920f1f7370a',
 '3b68d2152b010ea71fd90ba74ab573523ec48d4b',
 'fbd2ae61674d381eb6adcf5b905329fb9520cacb',
 '27e2d4fc64c5b7779d151f8760d43d8119121470',
 'bc795e8cc660a85b569a8002f3ead4dc45145844',
 '28f1848f8cb16b0a7f1fed74f44939255ad13845',
 'd00a2afb7062940198b834cdbbef0b170e5729b9',
 '8ee7c21875c1d8f1d9d38783e80a0c57ca41edf3',
 'b94ebb18238eaaf98146f9e480e06efb4007c3df',
 'bb9e1adc5c0aa36428c93240201e688b30001173',
 'c3e2e5b31ec3f86667ae87b9744c2712f614f4b5',
 'ceb8bc657d737e0297a3f0314790c6e1c5e9c160',
 '5ec0e3d7